In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
from pathlib import Path # pathlib 라이브러리 임포트

# --- 경로 설정 (상대 경로 방식으로 변경) ---
# 1. 현재 스크립트 파일 또는 작업 디렉토리를 기준으로 프로젝트의 최상위 폴더(Main)를 찾습니다.
try:
    # .py 파일로 실행할 때를 위한 기본 경로
    # 이 경우, 스크립트 파일이 프로젝트의 'Main' 폴더 내에 있다고 가정합니다.
    MAIN_DIR = Path(__file__).resolve().parent
except NameError:
    # Jupyter Notebook 등 __file__이 정의되지 않은 환경을 위한 대체 경로
    # 이 노트북 파일이 프로젝트의 'Main' 폴더 내에 있다고 가정합니다.
    MAIN_DIR = Path.cwd()

# 2. Main 폴더를 기준으로 Data와 Results 폴더의 경로를 설정합니다.
# 이렇게 하면 어떤 컴퓨터에서 실행해도 경로가 올바르게 설정됩니다.
DATA_DIR = MAIN_DIR / "Data"
RESULTS_DIR = MAIN_DIR / "Results"

# 3. 최종 파일 경로들을 정의합니다.
RAW_DATA_DIR = DATA_DIR / "Raw data"
OPTIMAL_WEIGHTS_SAVE_PATH = RESULTS_DIR / "optimized_weights_all_periods.xlsx"

# --- 종목 지정 ---
STOCKS = [
    'SPY', 'QQQ', 'EFA', 'EEM', 'SSO', 'TQQQ', # 주식
    'TLT', 'IEF', 'SHY',       # 채권
    'GLD', 'DBC',              # 원자재
    'BTC-USD', 'ETH-USD', 'XRP-USD'                  # 암호화폐
]

# --- 기간 설정 ---
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=10*365)

def get_data(tickers, start, end, save_local_path=None):
    """지정된 티커의 주가 데이터를 다운로드하고, 'Adj Close'를 사용하며, 컬럼 순서를 재정렬합니다."""
    print(f"'{tickers}' 종목 데이터 다운로드 시도 중...")
    data = yf.download(tickers, start=start, end=end, auto_adjust=False)
    if data.empty: raise ValueError("데이터 다운로드 실패")
    
    price_data = data['Adj Close']
    price_data = price_data.dropna()
    
    present_tickers = [ticker for ticker in tickers if ticker in price_data.columns]
    price_data = price_data[present_tickers]
    print("데이터프레임 컬럼을 STOCKS 리스트 순서대로 재정렬했습니다.")

    if price_data.empty: raise ValueError("유효한 데이터가 없습니다.")

    if save_local_path:
        # 파일 경로에서 디렉토리 부분만 추출하여 생성
        save_dir = os.path.dirname(save_local_path)
        os.makedirs(save_dir, exist_ok=True)
        price_data.to_csv(save_local_path)
        print(f"수정 종가 데이터가 로컬에 저장되었습니다: {save_local_path}")
    return price_data

def calculate_returns(data, period='daily'):
    """주가 데이터를 기반으로 수익률을 계산합니다."""
    if period == 'weekly': return data.resample('W-FRI').last().pct_change().dropna()
    if period == 'monthly': return data.resample('ME').last().pct_change().dropna()
    return data.pct_change().dropna()

def portfolio_performance(weights, returns, period_type):
    """포트폴리오 성과(수익률, 표준편차, 샤프비율)를 계산합니다."""
    periods_per_year = {'daily': 252, 'weekly': 52, 'monthly': 12}
    p_return = np.sum(returns.mean() * weights) * periods_per_year[period_type]
    p_std_dev = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * periods_per_year[period_type], weights)))
    sharpe = p_return / p_std_dev if p_std_dev > 0 else 0
    return p_return, p_std_dev, sharpe

# --- 최적화 목표 함수들 ---
def neg_sharpe_ratio(weights, returns, period_type):
    """최적화를 위해 샤프비율에 음수를 취합니다."""
    return -portfolio_performance(weights, returns, period_type)[2]

def portfolio_volatility(weights, returns, period_type):
    """포트폴리오의 연간 변동성(표준편차)을 계산합니다."""
    return portfolio_performance(weights, returns, period_type)[1]

# --- 최적화 실행 함수들 ---
def optimize_max_sharpe(returns, period_type, weight_cap=1.0):
    """샤프비율을 최대화하는 최적의 포트폴리오 가중치를 찾습니다."""
    num_assets = len(returns.columns)
    bounds = tuple((0, weight_cap) for _ in range(num_assets))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    init_guess = np.array([1/num_assets] * num_assets)
    
    optimal_results = minimize(neg_sharpe_ratio, init_guess, args=(returns, period_type),
                               method='SLSQP', bounds=bounds, constraints=constraints)
    return optimal_results.x

def optimize_min_volatility(returns, period_type, target_return=None):
    """
    변동성을 최소화하는 포트폴리오를 찾습니다.
    target_return이 주어지면, 해당 수익률을 만족하는 조건 하에서 변동성을 최소화합니다.
    """
    num_assets = len(returns.columns)
    init_guess = np.array([1/num_assets] * num_assets)
    bounds = tuple((0, 1) for _ in range(num_assets))
    
    # 제약 조건 설정
    cons = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]
    if target_return is not None:
        # 목표 수익률 제약 추가
        periods_per_year = {'daily': 252, 'weekly': 52, 'monthly': 12}
        cons.append({'type': 'ineq', 'fun': lambda weights: np.sum(returns.mean() * weights) * periods_per_year[period_type] - target_return})

    optimal_results = minimize(portfolio_volatility, init_guess, args=(returns, period_type),
                               method='SLSQP', bounds=bounds, constraints=cons)
    return optimal_results.x

def calculate_risk_parity_weights(returns):
    """위험 패리티 포트폴리오의 가중치를 계산합니다."""
    vols = returns.std()
    inv_vols = 1.0 / vols
    return (inv_vols / inv_vols.sum()).values

# --- 메인 실행 흐름 ---
if __name__ == "__main__":
    downloaded_data_path = RAW_DATA_DIR / 'downloaded_stock_prices.csv'
    stock_data = pd.DataFrame()
    try:
        stock_data = get_data(STOCKS, START_DATE, END_DATE, save_local_path=downloaded_data_path)
    except Exception as e:
        print(f"오류: 데이터 다운로드 중 문제 발생: {e}")

    if not stock_data.empty:
        results = {}
        final_tickers = stock_data.columns.tolist()
        daily_returns = calculate_returns(stock_data, period='daily')

        # 1. 샤프지수 최대화 모델 (제한 없음)
        for period_type in ['daily', 'weekly', 'monthly']:
            print(f"\n--- {period_type.capitalize()} (Max Sharpe) 분석 시작 ---")
            returns = calculate_returns(stock_data, period=period_type)
            if returns.empty: continue
            results[period_type] = {'weights': optimize_max_sharpe(returns, period_type)}

        # 2. 위험 패리티 모델
        print("\n--- Risk Parity 분석 시작 ---")
        if not daily_returns.empty:
            results['risk_parity'] = {'weights': calculate_risk_parity_weights(daily_returns)}

        # 3. 샤프지수 최대화 (최대 비중 30% 제한) 모델
        print("\n--- Daily (Max Sharpe, 30% Cap) 분석 시작 ---")
        if not daily_returns.empty:
            results['daily_30_cap'] = {'weights': optimize_max_sharpe(daily_returns, 'daily', weight_cap=0.30)}
            
        # 4. 최소 분산 모델
        print("\n--- Minimum Variance 분석 시작 ---")
        if not daily_returns.empty:
            results['min_variance'] = {'weights': optimize_min_volatility(daily_returns, 'daily')}

        # 5. 목표 수익률 하 위험 최소화 모델
        TARGET_RETURN = 0.18 # 연 18% 목표
        print(f"\n--- Target Return ({TARGET_RETURN*100}%) 분석 시작 ---")
        if not daily_returns.empty:
            results[f'target_return_{int(TARGET_RETURN*100)}'] = {'weights': optimize_min_volatility(daily_returns, 'daily', target_return=TARGET_RETURN)}

        # --- 모든 모델의 가중치를 하나의 Excel 파일에 저장 ---
        if results:
            os.makedirs(RESULTS_DIR, exist_ok=True)
            with pd.ExcelWriter(OPTIMAL_WEIGHTS_SAVE_PATH, engine='openpyxl') as writer:
                for model_name, data in results.items():
                    weights_df = pd.DataFrame({'Ticker': final_tickers, 'Optimal_Weight': data['weights']})
                    weights_df.to_excel(writer, sheet_name=f'{model_name}_weights', index=False)
                    print(f"'{model_name}' 모델 가중치 저장 완료.")
            print(f"\n모든 모델의 최적 가중치가 저장되었습니다: {OPTIMAL_WEIGHTS_SAVE_PATH}")


'['SPY', 'QQQ', 'EFA', 'EEM', 'SSO', 'TQQQ', 'TLT', 'IEF', 'SHY', 'GLD', 'DBC', 'BTC-USD', 'ETH-USD', 'XRP-USD']' 종목 데이터 다운로드 시도 중...



*********************100%***********************]  14 of 14 completed

데이터프레임 컬럼을 STOCKS 리스트 순서대로 재정렬했습니다.
수정 종가 데이터가 로컬에 저장되었습니다: C:\Users\012oov\Documents\012\Quant\1. MPT_Back\Main\MPT_Back\Data\Raw data\downloaded_stock_prices.csv

--- Daily (Max Sharpe) 분석 시작 ---

--- Weekly (Max Sharpe) 분석 시작 ---

--- Monthly (Max Sharpe) 분석 시작 ---

--- Risk Parity 분석 시작 ---

--- Daily (Max Sharpe, 30% Cap) 분석 시작 ---

--- Minimum Variance 분석 시작 ---

--- Target Return (18.0%) 분석 시작 ---
'daily' 모델 가중치 저장 완료.
'weekly' 모델 가중치 저장 완료.
'monthly' 모델 가중치 저장 완료.
'risk_parity' 모델 가중치 저장 완료.
'daily_30_cap' 모델 가중치 저장 완료.
'min_variance' 모델 가중치 저장 완료.
'target_return_18' 모델 가중치 저장 완료.

모든 모델의 최적 가중치가 저장되었습니다: C:\Users\012oov\Documents\012\Quant\1. MPT_Back\Main\MPT_Back\Results\optimized_weights_all_periods.xlsx
